In [ ]:
import pandas as pd
from matplotlib.pyplot import figure
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import plotly.express as px

import torch
import optuna
import torch_geometric.transforms as T
from sklearn.neural_network import MLPClassifier 
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from torch_geometric.data import NeighborSampler
from torch.optim import lr_scheduler
import torch.optim as optim
import collections
import numpy as np
from modules.model import Net
from sklearn.metrics import f1_score
from modules.sampling import Sampler, SamplerContextMatrix, SamplerRandomWalk,SamplerFactorization,SamplerAPP
from datetime import datetime
import random
from torch_geometric.data import GraphSAINTNodeSampler
import pickle
import os
from torch_geometric.data import Data
import collections
from torch_geometric.datasets import Planetoid,WikipediaNetwork,Actor,WebKB

In [ ]:
import pandas as pd
import collections
df=pd.read_csv('data_analysis_realdata.csv')
df=df.drop(columns=['Unnamed: 0'])
df

In [ ]:
(df[(df['dataset']=='Cora') & (df['conv']=='GCN')]['test acc micro'].max(),df[(df['dataset']=='Cora') & (df['conv']=='GCN')]['test acc micro'].std())

In [ ]:
(df[(df['dataset']=='Citeseer') & (df['conv']=='GCN')]['test acc micro'].max(),df[(df['dataset']=='Citeseer') & (df['conv']=='GCN')]['test acc micro'].std())

In [ ]:
(df[(df['dataset']=='Cornell') & (df['conv']=='GCN')]['test acc micro'].max(),df[(df['dataset']=='Cornell') & (df['conv']=='GCN')]['test acc micro'].std())

In [ ]:
(df[(df['dataset']=='chameleon') & (df['conv']=='GCN')]['test acc micro'].max(), df[(df['dataset']=='chameleon') & (df['conv']=='GCN')]['test acc micro'].std())

In [ ]:
(df[(df['dataset']=='Wisconsin') & (df['conv']=='GCN') ]['test acc micro'].max(),df[(df['dataset']=='Wisconsin') & (df['conv']=='GCN') ]['test acc micro'].std())

In [ ]:
(df[(df['dataset']=='Texas') & (df['conv']=='GCN') ]['test acc micro'].max() ,df[(df['dataset']=='Texas') & (df['conv']=='GCN') ]['test acc micro'].std())

In [ ]:
losses=list(dict(collections.Counter(df['loss'])).keys())

attrs=['label assortativity','feature assortativity', 'cluster coefficient', 'average shortest path','average degree']


In [ ]:
font = {'size' : 25}
matplotlib.rc('font', **font)

# CORRELATIONS 

In [ ]:
from scipy.stats import kendalltau as Kcorr
from scipy.stats import pearsonr as Pcorr

In [ ]:

df=pd.read_csv('final_data.csv')
df=df.drop(columns=['Unnamed: 0'])

losses = list(dict(collections.Counter(df['loss'])).keys())

su=0
ff=0
for i,loss1 in enumerate(losses):
    for loss2 in losses[i+1:]:
       # print(loss1,loss2)
        su+=1
        x=df[(df['loss']==loss1) & (df['conv'] == 'GCN' )]['test acc micro']
        y=df[(df['loss']==loss2) & (df['conv'] == 'GCN' )]['test acc micro']
        if Pcorr(x,y)[0]<0.98:
            ff+=1   
print(ff,su)
            

# COMPARISON OF DENSITY OF BENCHMARK SPACE

In [ ]:
df_2 = pd.read_excel('benchmark_space_ideal.xlsx')

In [ ]:
import pandas as pd
import collections
df=pd.read_csv('final_data.csv')
df=df.drop(columns=['Unnamed: 0'])

In [ ]:
font = {'size' : 20}
matplotlib.rc('font', **font)

In [ ]:
fig2=px.parallel_coordinates(df,dimensions=['label assortativity', 'feature assortativity', 'cluster coefficient', 'average shortest path', 'average degree'])
fig2.update_layout(
    font=dict(
        size=22,  # Set the font size here
        color = 'black'
    )
)

fig2.show()

In [ ]:
fig2=px.parallel_coordinates(df_2,dimensions=['label assortativity', 'feature assortativity', 'cluster coefficient', 'average shortest path', 'average degree'])
fig2.update_layout(
    font=dict(
        size=22,  # Set the font size here
        color = 'black'
    )
)

fig2.show()

# heatmaps

In [ ]:
df_analysis = df.groupby(['group','label assortativity']).mean()['test acc micro']
df_analysis = df_analysis.unstack('label assortativity')
df_analysis['newlevel'] = 'label assortativity'
df_analysis=df_analysis.set_index('newlevel', append=True).unstack('newlevel')

for attr in attrs:
    df_1 = df.groupby(['group',attr]).mean()['test acc micro']
    df_1 = df_1.unstack(attr)
    df_1['newlevel'] = attr
    df_1 = df_1.set_index('newlevel', append=True).unstack('newlevel')
    
    df_analysis = pd.merge(df_analysis,df_1,on=['group'],how='left')
    

In [ ]:
df_analysis

In [ ]:
for attr in ['label assortativity']+attrs:
    df_1 = df.groupby(['group',attr]).mean()['test acc macro']
    df_1 = df_1.unstack(attr)
    sns.heatmap(df_1,cmap='GnBu')
    plt.title('macro')
    plt.show()

# Global Ranking

In [ ]:
df_analysis = df.groupby(['group','loss','label assortativity']).std()['test acc micro']

df_analysis = df_analysis.unstack('label assortativity')
df_analysis['newlevel'] = 'label assortativity'
df_analysis=df_analysis.set_index('newlevel', append=True).unstack('newlevel')

for attr in attrs[1:]:
    df_1 = df.groupby(['group','loss',attr]).std()['test acc micro']
    df_1 = df_1.unstack(attr)
    df_1['newlevel'] = attr
    df_1 = df_1.set_index('newlevel', append=True).unstack('newlevel')
    
    df_analysis = pd.merge(df_analysis,df_1,on=['group','loss'],how='left')

df_analysis=df_analysis.round(decimals=2)
df_analysis

In [ ]:
df_analysis = df.groupby(['group','loss','label assortativity']).mean()['test acc micro']

df_analysis = df_analysis.unstack('label assortativity')
df_analysis['newlevel'] = 'label assortativity'
df_analysis=df_analysis.set_index('newlevel', append=True).unstack('newlevel')

for attr in attrs[1:]:
    df_1 = df.groupby(['group','loss',attr]).mean()['test acc micro']
    df_1 = df_1.unstack(attr)
    df_1['newlevel'] = attr
    df_1 = df_1.set_index('newlevel', append=True).unstack('newlevel')
    
    df_analysis = pd.merge(df_analysis,df_1,on=['group','loss'],how='left')

df_analysis=df_analysis.round(decimals=2)
df_analysis

# la=0.1

In [ ]:
df_analysis = df[(df['label assortativity']==0.1) ]

df_analysis = df_analysis.groupby(['group','loss','feature assortativity']).mean()['test acc micro']
df_analysis = df_analysis.unstack('feature assortativity')
df_analysis['newlevel'] = 'feature assortativity'
df_analysis=df_analysis.set_index('newlevel', append=True).unstack('newlevel')

for attr in attrs[2:]:
    df_1 =  df[(df['label assortativity']==0.1) ].groupby(['group','loss',attr]).mean()['test acc micro']
    df_1 = df_1.unstack(attr)
    df_1['newlevel'] = attr
    df_1 = df_1.set_index('newlevel', append=True).unstack('newlevel')
    
    df_analysis = pd.merge(df_analysis,df_1,on=['group','loss'],how='left')

df_analysis=df_analysis.round(decimals=2)
df_analysis

In [ ]:
df_analysis = df[(df['label assortativity']==0.1)]

df_analysis = df_analysis.groupby(['group','loss','feature assortativity']).std()['test acc micro']
df_analysis = df_analysis.unstack('feature assortativity')
df_analysis['newlevel'] = 'feature assortativity'
df_analysis=df_analysis.set_index('newlevel', append=True).unstack('newlevel')

for attr in attrs[2:]:
    df_1 =  df[(df['label assortativity']==0.1) ].groupby(['group','loss',attr]).std()['test acc micro']
    df_1 = df_1.unstack(attr)
    df_1['newlevel'] = attr
    df_1 = df_1.set_index('newlevel', append=True).unstack('newlevel')
    
    df_analysis = pd.merge(df_analysis,df_1,on=['group','loss'],how='left')

df_analysis=df_analysis.round(decimals=2)
df_analysis

# la=0.5

In [ ]:
df_analysis = df[(df['label assortativity']==0.5)]
df_analysis = df_analysis.groupby(['group','loss','feature assortativity']).mean()['test acc micro']
df_analysis = df_analysis.unstack('feature assortativity')
df_analysis['newlevel'] = 'feature assortativity'
df_analysis=df_analysis.set_index('newlevel', append=True).unstack('newlevel')

for attr in attrs[2:]:
    df_1 = df[(df['label assortativity']==0.5)].groupby(['group','loss',attr]).mean()['test acc micro']
    df_1 = df_1.unstack(attr)
    df_1['newlevel'] = attr
    df_1 = df_1.set_index('newlevel', append=True).unstack('newlevel')
    
    df_analysis = pd.merge(df_analysis,df_1,on=['group','loss'],how='left')
df_analysis=df_analysis.round(decimals=2)
df_analysis

In [ ]:
df_analysis = df[(df['label assortativity']==0.5)]
df_analysis = df_analysis.groupby(['group','loss','feature assortativity']).std()['test acc micro']
df_analysis = df_analysis.unstack('feature assortativity')
df_analysis['newlevel'] = 'feature assortativity'
df_analysis=df_analysis.set_index('newlevel', append=True).unstack('newlevel')

for attr in attrs[2:]:
    df_1 = df[(df['label assortativity']==0.5) ].groupby(['group','loss',attr]).std()['test acc micro']
    df_1 = df_1.unstack(attr)
    df_1['newlevel'] = attr
    df_1 = df_1.set_index('newlevel', append=True).unstack('newlevel')
    
    df_analysis = pd.merge(df_analysis,df_1,on=['group','loss'],how='left')
df_analysis=df_analysis.round(decimals=2)
df_analysis

# la=0.9

In [ ]:
df_analysis = df[(df['label assortativity']==0.9) ]
df_analysis = df_analysis.groupby(['group','loss','feature assortativity']).mean()['test acc micro']
df_analysis = df_analysis.unstack('feature assortativity')
df_analysis['newlevel'] = 'feature assortativity'
df_analysis=df_analysis.set_index('newlevel', append=True).unstack('newlevel')

for attr in attrs[2:]:
    df_1 = df[(df['label assortativity']==0.9)].groupby(['group','loss',attr]).mean()['test acc micro']
    df_1 = df_1.unstack(attr)
    df_1['newlevel'] = attr
    df_1 = df_1.set_index('newlevel', append=True).unstack('newlevel')
    
    df_analysis = pd.merge(df_analysis,df_1,on=['group','loss'],how='left')
df_analysis=df_analysis.round(decimals=2)
df_analysis.transpose()

In [ ]:
df_analysis = df[(df['label assortativity']==0.9) ]
df_analysis = df_analysis.groupby(['group','loss','feature assortativity']).std()['test acc micro']
df_analysis = df_analysis.unstack('feature assortativity')
df_analysis['newlevel'] = 'feature assortativity'
df_analysis=df_analysis.set_index('newlevel', append=True).unstack('newlevel')

for attr in attrs[2:]:
    df_1 = df[(df['label assortativity']==0.9) ].groupby(['group','loss',attr]).std()['test acc micro']
    df_1 = df_1.unstack(attr)
    df_1['newlevel'] = attr
    df_1 = df_1.set_index('newlevel', append=True).unstack('newlevel')
    
    df_analysis = pd.merge(df_analysis,df_1,on=['group','loss'],how='left')
df_analysis=df_analysis.round(decimals=2)
df_analysis

In [ ]:
import pandas as pd
import collections
df=pd.read_csv('final_data.csv')
df=df.drop(columns=['Unnamed: 0'])

In [ ]:
df_groupby=df[['conv','dataset','test acc micro']].groupby(['conv','dataset'],as_index=False).max()
df_merge = pd.merge(df_groupby,df,on=['conv','dataset','test acc micro'],how='left')
list_of_dicts=[]

for conv in ['GCN','GAT','SAGE']:
    orders = collections.Counter(df_merge[df_merge['conv']==conv]['loss'])
    list_of_dicts.append(orders)
new_dict=list_of_dicts[0]+list_of_dicts[1]+list_of_dicts[2]

global_rank=(sorted(new_dict.items(), key=lambda x: x[1], reverse=True))
rank_GCN=(sorted(list_of_dicts[0].items(), key=lambda x: x[1], reverse=True))
rank_GAT=(sorted(list_of_dicts[1].items(), key=lambda x: x[1], reverse=True))
rank_SAGE=(sorted(list_of_dicts[2].items(), key=lambda x: x[1], reverse=True))
print(rank_SAGE,rank_GCN,rank_GAT,global_rank)

In [ ]:
dict(rank_GCN)

# Line Graphs

In [ ]:

df=pd.read_csv('final_data.csv')
df=df.drop(columns=['Unnamed: 0'])

In [ ]:
for feat in features:
    for conv in ['GCN','GAT','SAGE']:
        plt.plot( list(df[ (df['conv'] == conv)].groupby(feat).mean().index),  df[(df['conv']==conv)].groupby(feat).mean()['test acc micro'],label=conv,marker='*', linewidth = 1)
    plt.legend()
    plt.title(feat)
    plt.show()
    

# boxplots 

In [ ]:
df_boxplots = df[(df['loss' ]=='VERSE_Adj')|(df['loss']=='HOPE_RPR') | (df['loss']=='LapEigen') | (df['loss']=='VERSE_SR') | (df['loss' ]=='DeepWalk')| (df['loss' ]=='Force2Vec')]
features=['label assortativity','feature assortativity','cluster coefficient','average shortest path','average degree']


# Statistically significant difference

In [ ]:
#ANOVA
losses=list(dict(collections.Counter(df['loss'])).keys())
import scipy.stats as stats
to_test = []
for loss in losses:
    to_test.append(df[df['loss']==loss]['test acc micro'])

fvalue, pvalue = stats.f_oneway(to_test[0],to_test[1],to_test[2],to_test[3],to_test[4],to_test[5],to_test[6],to_test[7],to_test[8],to_test[9],to_test[10],to_test[11],to_test[12])
#fvalue, pvalue = stats.f_oneway(df[df['loss']=='HOPE_Katz']['test acc micro'],df[df['loss']=='HOPE_CN']['test acc micro'], df[df['loss']=='HOPE_RPR']['test acc micro'], df[df['loss']=='LapEigen']['test acc micro'],df[df['loss']=='LINE']['test acc micro'],df[df['loss']=='VERSE_PPR']['test acc micro'], df[df['loss']=='GraphFactorization']['test acc micro'], df[df['loss']=='HOPE_AA']['test acc micro'], df[df['loss']=='Node2Vec']['test acc micro'], df[df['loss']=='VERSE_Adj']['test acc micro'],df[df['loss']=='DeepWalk']['test acc micro'],df[df['loss']=='APP']['test acc micro'],df[df['loss']=='VERSE_SR']['test acc micro'])


In [ ]:
#внутри группы GF
fvalue, pvalue = stats.f_oneway(to_test[0],to_test[1],to_test[2],to_test[6],to_test[7])
print(pvalue<0.05)

In [ ]:
#внутри группы RandomWalks
fvalue, pvalue = stats.f_oneway(to_test[1],to_test[8])
print(pvalue<0.05)

In [ ]:
#внутри группы matrix sim
fvalue, pvalue = stats.f_oneway(to_test[4],to_test[5],to_test[9],to_test[11],to_test[12])
print(pvalue<0.05)

In [ ]:
import numpy as np
from scipy.stats import ttest_ind

losses_2=['LINE','VERSE_SR','VERSE_PPR','VERSE_Adj','APP']
for i,loss1 in enumerate(losses_2):
    for loss2 in losses_2[i+1:]:
       # print(loss1,loss2)
       # print(loss)
        v1 = df[df['loss']==loss1]['test acc micro']
        v2 = df[df['loss']==loss2]['test acc micro']

        res = ttest_ind(v1, v2)

        if (res.pvalue <= 0.05):
            print(loss1,loss2)

# ВЫБИВАЕТСЯ ТОЛЬКО VERSE ADJ 